In [1]:
import argparse
import torch
from torch_geometric import utils
from torch_geometric.datasets import TUDataset
from torch_geometric import data

In [2]:
dataset = TUDataset(root = "molecule_dataset", name = "MUTAG")

In [3]:
maxNodes = 0
for i in range(len(dataset)):
    n = dataset[i].num_nodes
    if (n > maxNodes):
        maxNodes = n

print(maxNodes)

28


In [4]:
testMol = dataset[0]
s = testMol.edge_stores[0]
s

{'edge_index': tensor([[ 0,  0,  1,  1,  2,  2,  3,  3,  3,  4,  4,  4,  5,  5,  6,  6,  7,  7,
          8,  8,  8,  9,  9,  9, 10, 10, 11, 11, 12, 12, 12, 13, 13, 14, 14, 14,
         15, 16],
        [ 1,  5,  0,  2,  1,  3,  2,  4,  9,  3,  5,  6,  0,  4,  4,  7,  6,  8,
          7,  9, 13,  3,  8, 10,  9, 11, 10, 12, 11, 13, 14,  8, 12, 12, 15, 16,
         14, 14]]), 'x': tensor([[1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0

In [5]:
print(s['edge_index'].size())   # 2 x E edges
print(s['x'].size())            # N nodes x 7 node features
print(s['edge_attr'].size())    #E edges X 4 edge features

torch.Size([2, 38])
torch.Size([17, 7])
torch.Size([38, 4])


GOAL:
1) create ordered map of (edge_type, vector_idx)
2) create function to use map to identify vector location

In [6]:
# MAKE MAP
vector_map = dict()

pos_ctr = 0
for i in range(28):
    for j in range(i):
        vector_map[str(j) + "-" + str(i)] = pos_ctr
        pos_ctr += 1

In [14]:
# VECTOR FINAL DIMENSIONS:

# adj matrix = len(vector_map)
dim1 = len(vector_map)

# feature matrix = (max nodes) * 7
dim2 = maxNodes * 7

finalDim = dim1 + dim2
finalDim

574

In [17]:
# FUNCTION TO MAP EDGES TO VECTOR + CONCATS
def get_input_vector(input: data.Data):
    adj_vector = torch.zeros(size = (len(vector_map), 1))
    indices = input.edge_stores[0]['edge_index']
    features = input.edge_stores[0]['x']

    e1 = indices[0].tolist()
    e2 = indices[1].tolist()
    
    # fill max atom adj matrix w/ connections
    for i in range(len(e1)):
        key = str(e1[i]) + "-" + str(e2[i])
        if key in vector_map:
            idx = vector_map[key]
            adj_vector[idx] = 1

    # reshape feature matrix into vector
    feature_vec = torch.reshape(s['x'], shape = (-1,1))

    # concatenate vectorized feature & adj matrices
    concat_vector = torch.cat((adj_vector, feature_vec))

    prevSize = concat_vector.size(dim = 0)
    padSize = 574 - prevSize
    padding = (0, 0, 0, padSize)

    # pad with 0s for consistency
    concat_vector = torch.nn.functional.pad(concat_vector, padding, value = 0)

    return concat_vector


In [ ]:
# EXAMPLE
get_input_vector(dataset[0])

TODO:
- add padding to vector (in function)
    - CHECK -- does filling vector w/ zeros work w/ masking layer? or is different type required
    - CHECK -- 574 x 1 desired or 1 x 574?
- create dataset class & dataloader (?)
- look into masking layer for GNN